# 🧠 Bayesian-AI Learning Dashboard: Preflight & Quick Learn

**Objective:** Validate environment and perform rapid learning simulations before full training cycles.
**Workflow:** Preflight -> Data Check -> 3-Day Discrete Simulation -> Full Learn Cycle

## 1. Preflight Checks ✈️
Verify Operational Mode, Environment, and CUDA Audit.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output
from numba import cuda

# Add root to path
current_dir = Path(os.getcwd())
project_root = current_dir
while not (project_root / 'requirements.txt').exists():
    parent = project_root.parent
    if parent == project_root:
        break
    project_root = parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
print(f"Project Root: {project_root}")

from config.settings import OPERATIONAL_MODE, RAW_DATA_PATH
from training.orchestrator import get_data_source, load_data_from_directory

# 1. Check Operational Mode
print(f"\nOperational Mode: {OPERATIONAL_MODE}")
if OPERATIONAL_MODE != "LEARNING":
    print("🔴 WARNING: System should be in 'LEARNING' mode for this dashboard.")
else:
    print("🟢 Mode Checked")

# 2. Check CUDA
try:
    if cuda.is_available():
        print(f"🟢 CUDA Available: {cuda.detect()}")
    else:
        print("🟡 CUDA Not Available (using CPU)")
except Exception as e:
    print(f"🔴 CUDA Check Failed: {e}")

# 3. Run Deep Audit
import cuda_modules.hardened_verification as hv
print("\nRunning 3-Stage CUDA Audit...")
try:
    success = hv.run_audit()
    if success:
        print("🟢 Audit Passed! (See CUDA_Debug.log)")
    else:
        print("🔴 Audit Failed! (See CUDA_Debug.log)")
except Exception as e:
    print(f"🔴 Execution Error: {e}")

## 2. Data Setup & Verification 📊
Ensure `DATA/RAW` contains valid data and visualize a sample.

In [ ]:
try:
    print(f"Loading data from {RAW_DATA_PATH}...")
    # Load full dataset into memory for slicing
    full_data = load_data_from_directory(str(project_root / RAW_DATA_PATH))
    
    print(f"🟢 Data Loaded! Rows: {len(full_data)}")
    
    # Ensure timestamp conversion
    if 'timestamp' in full_data.columns:
        full_data['datetime'] = pd.to_datetime(full_data['timestamp'], unit='s' if full_data['timestamp'].dtype == 'float64' else None)
    
    # Show date range
    min_date = full_data['datetime'].min()
    max_date = full_data['datetime'].max()
    print(f"Date Range: {min_date} to {max_date}")
    
    # Visualize Sample
    plot_df = full_data.head(1000)
    fig = go.Figure(data=[go.Candlestick(x=plot_df['datetime'],
                open=plot_df['price'], high=plot_df['price'],
                low=plot_df['price'], close=plot_df['price'])])
    fig.update_layout(title='Price Sample (First 1000 ticks)', xaxis_rangeslider_visible=False)
    fig.show()
    
except Exception as e:
    print(f"🔴 Data Load Failed: {e}")
    full_data = None

## 3. Quick Learn: 3 Discrete Day Simulation 🎲
Randomly select 3 days and run isolated learning simulations to verify logic and win rates.

In [ ]:
import random
from training.orchestrator import TrainingOrchestrator

def run_quick_learn():
    if full_data is None:
        print("🔴 No data loaded.")
        return

    # Identify unique days
    unique_days = full_data['datetime'].dt.date.unique()
    
    if len(unique_days) < 3:
        print(f"🟡 Not enough data for 3 discrete days. Found: {len(unique_days)}")
        selected_days = unique_days
    else:
        selected_days = sorted(random.sample(list(unique_days), 3))
    
    print(f"Selected Days: {selected_days}")
    
    results = []
    
    for day in selected_days:
        print(f"\n--- Simulating {day} ---")
        day_mask = full_data['datetime'].dt.date == day
        day_data = full_data[day_mask].copy()
        
        # Initialize Orchestrator for this chunk
        # We use a temporary output dir to not overwrite main model
        orch = TrainingOrchestrator(
            asset_ticker='MNQ', 
            data=day_data, 
            output_dir='debug_outputs/quick_learn'
        )
        
        # Run 1 iteration (simulation of that day)
        # Params dict ensures we don't save to main pkl
        metrics = orch.run_training(iterations=1, params={'mode': 'quick_learn'})
        metrics['date'] = day
        results.append(metrics)
        
    # Summary
    print("\n=== Quick Learn Summary ===")
    res_df = pd.DataFrame(results)
    display(res_df[['date', 'total_trades', 'win_rate', 'pnl']])

btn_quick = widgets.Button(description="Run 3-Day Sim", button_style='info')
out_quick = widgets.Output()

def on_quick_click(b):
    out_quick.clear_output()
    with out_quick:
        run_quick_learn()

btn_quick.on_click(on_quick_click)
display(btn_quick, out_quick)

## 4. Full Learning Cycle 🚀
Execute the full training pipeline on all available data.

In [ ]:
import subprocess

def run_full_training(iterations=10):
    print(f"Starting Full Training ({iterations} iterations)...")
    cmd = [
        "python", "training/orchestrator.py",
        "--data-dir", "DATA/RAW",
        "--iterations", str(iterations),
        "--output", "models/"
    ]
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    while True:
        line = process.stdout.readline()
        if not line and process.poll() is not None:
            break
        if line:
            print(line.strip())
            
    if process.returncode == 0:
        print("\n✅ Full Training Complete!")
    else:
        print(f"\n❌ Training Failed (Code {process.returncode})")

btn_full = widgets.Button(description="Execute Full Learn", button_style='danger')
out_full = widgets.Output()

def on_full_click(b):
    out_full.clear_output()
    with out_full:
        # Warning prompt could go here, for now direct execution
        run_full_training()

btn_full.on_click(on_full_click)
display(btn_full, out_full)

## 5. Result Analysis 📈
Analyze the probability tables generated.

In [ ]:
import pickle

def inspect_table(path, title):
    if not os.path.exists(path):
        print(f"⚪ {title}: File not found ({path})")
        return
    
    with open(path, 'rb') as f:
        data = pickle.load(f)
    table = data['table'] if isinstance(data, dict) and 'table' in data else data.table
    print(f"🟢 {title}: {len(table)} states learned.")

print("--- Model Status ---")
inspect_table('debug_outputs/quick_learn/probability_table.pkl', "Quick Learn Temp")
inspect_table('models/probability_table.pkl', "Main Model")